In [1]:
import sddk
import pandas as pd
import tabulate
pd.options.display.max_columns = 1000 # to see all columns
import json
import re
import geopandas as gpd
from shapely.geometry import Point
from functools import partial
from shapely.ops import transform
import pyproj
import warnings
warnings.filterwarnings('ignore')
import numpy as np
from scipy.spatial import cKDTree
import matplotlib.pyplot as plt

from sklearn.feature_extraction.text import CountVectorizer
from sklearn.feature_extraction.text import TfidfVectorizer

from sklearn.model_selection import train_test_split
from sklearn.svm import SVC
from sklearn.metrics import confusion_matrix

import pickle

import nltk
from nltk.util import bigrams

In [2]:
EDHCSg = sddk.read_file("EDHCSg.geojson", "gdf", "a9237c5ea642d4714bcdefb03f70a1f4")

reading file located in a public folder
The history saving thread hit an unexpected error (OperationalError('database or disk is full',)).History will not be written to the database.


In [3]:
# to train the model, we will work with a subset of the for which we have both the EDH and EDCS attributes
# we will call it EDHg
EDH_overlap_all = EDHCSg[(EDHCSg["EDH-ID"].notnull()) & (EDHCSg["EDCS-ID"].notnull())] 
EDH_overlap_all.groupby("type_of_inscription_clean").size().sort_values(ascending=False)

type_of_inscription_clean
epitaph                            21520
votive inscription                 11728
NULL                                3745
owner/artist inscription            3340
honorific inscription               3003
building/dedicatory inscription     2561
mile-/leaguestone                   1307
identification inscription           850
acclamation                          287
defixio                              269
list                                 246
military diploma                     209
label                                194
boundary inscription                 175
elogium                              132
letter                               119
public legal inscription             109
seat inscription                      42
private legal inscription             36
prayer                                18
assignation inscription               15
calendar                              10
adnuntiatio                            1
dtype: int64

In [4]:
EDH_overlap = EDH_overlap_all[~EDH_overlap_all["type_of_inscription_clean"].str.contains("NULL")]

Now we can focus on some EDCS attributes (i.e. firt 28 columns) which might be good predictors of `type_of_inscription_clean` in EDH. First, look at `status_list`:

In [5]:
EDH_overlap.groupby("type_of_inscription_clean").size().sort_values(ascending=False)

type_of_inscription_clean
epitaph                            21520
votive inscription                 11728
owner/artist inscription            3340
honorific inscription               3003
building/dedicatory inscription     2561
mile-/leaguestone                   1307
identification inscription           850
acclamation                          287
defixio                              269
list                                 246
military diploma                     209
label                                194
boundary inscription                 175
elogium                              132
letter                               119
public legal inscription             109
seat inscription                      42
private legal inscription             36
prayer                                18
assignation inscription               15
calendar                              10
adnuntiatio                            1
dtype: int64

In [6]:
# alternative, treat whole dataset as a sample
EDH_sampled = EDH_overlap

In [7]:
y = EDH_sampled["type_of_inscription_clean"].tolist()

# Build a SVM model combining `status_list` & `clean_text_interpretive_word`

next step would be to build a model combining the information from the `status_list` with an informatation from the text of the inscription (i.e. `clean_text_interpretive_word`).

In [8]:
EDH_sampled.head(5)

,EDCS-ID,publication,province,province_list,place,place_list,end_yr_list,notes_dating,status_list,inscr_type,status_notation,inscr_process,notes_references,notes_comments,inscription,inscription_stripped_final,Links,dating from,dating to,status,Latitude,Longitude,photo,Material,Comment,EDH-ID,language_EDCS,clean_text_interpretive_word_EDCS,responsible_individual,type_of_inscription,letter_size,literature,work_status,height,diplomatic_text,people,depth,material,type_of_monument,province_label,width,transcription,country,uri,findspot_ancient,last_update,modern_region,findspot_modern,edh_geography_uri,commentary,trismegistos_uri,external_image_uris,fotos,idno_tm,placenames_refs,text_edition,origdate_text,layout_execution,layout_execution_text,support_objecttype,support_objecttype_text,support_material,support_material_text,support_decoration,keywords_term,keywords_term_text,type_of_inscription_clean,type_of_inscription_certainty,height_cm,width_cm,depth_cm,material_clean,type_of_monument_clean,type_of_monument_certainty,province_label_clean,province_label_certainty,country_clean,country_certainty,findspot_ancient_clean,findspot_ancient_certainty,modern_region_clean,modern_region_certainty,findspot_modern_clean,findspot_modern_certainty,findspot_clean,findspot_certainty,origdate_text_clean,clean_text_conservative,clean_text_interpretive_sentence,findspot,year_of_find,present_location,religion,geography,social_economic_legal_history,military,not_after,language,not_before,coordinates,clean_text_interpretive_word,urban_context,within_rome,nearest_city,city_id_hanson,city_pop_est,city_geometry,nearest_city_type,nearest_city_dist,type_of_inscription_auto,type_of_inscription_auto_prob,geometry
0,EDCS-03700724,"ZPE-108-159 = Thesprotia 00001 = AE 1993, 0140...",Achaia,Achaia,Agios Athanasios / Photike,"['Agios Athanasios', 'Photike']",313,,"['Augusti/Augustae', 'litterae erasae', 'ordo ...",tituli honorarii,"['Augusti/Augustae', 'ordo equester', 'tria no...",litterae erasae,,,Fortissimo et Piis/simo Caesari d(omino) n(ost...,Fortissimo et Piis/simo Caesari d(omino) n(ost...,http://db.edcs.eu/epigr/partner.php?s_language...,309.0,313,Augusti/Augustae; litterae erasae; ordo equest...,39.451218,20.766767,http://db.edcs.eu/epigr/bilder.php?bilder.php?...,None,None,HD052964,None,Fortissimo et Piissimo Caesari domino nostro G...,Cowey,honorific inscription,3-5.3 cm,"AE 1993, 1406.; V. Papadopoulou, AD 43 B, 1988...",checked with photo,99 cm,FORTISSIMO ET PIIS / SIMO CAESARI D N / GAL VA...,"[{'name': 'Gal. Val. [[Maximiano]]', 'nomen': ...",67 cm,None,statue base,Epirus,67 cm,Fortissimo et piis/simo Caesari d(omino) n(ost...,Greece,https://edh-www.adw.uni-heidelberg.de/edh/insc...,Photike,2014-09-16,Ípeiros,Paramythía,https://edh-www.adw.uni-heidelberg.de/edh/geog...,None,https://www.trismegistos.org/text/121715,{},{},121715,"['http://www.trismegistos.org/place/000655', '...",Fortissimo et piissimo Caesari...,309 AD – 313 AD,21,unbestimmt,57.0,Statuenbasis,138,unbestimmt,1000.0,69,Ehreninschrift,honorific inscription,Certain,99.0,67.0,67.0,NULL,statue base,Certain,Epirus,Certain,Greece,Certain,Photike,Certain,Ípeiros,Certain,Paramythía,Certain,Agios Athanasios,Certain,309 AD – 313 AD,Fortissimo et piissimo Caesari d n Gal Val P F...,Fortissimo et piissimo Caesari domino nostro G...,Agios Athanasios,None,Fragma Kalama,None,data available,None,None,313.0,Latin,309.0,"[20.7668, 39.4512]",Fortissimo et piissimo Caesari domino nostro G...,rest,False,Dodona,31,1000.0,"[20.787767, 39.546432]",minor,0.097513,honorific inscription,1.0,POINT (20.76680 39.45120)
1,EDCS-03300852,"AE 1995, 01409",Achaia,Achaia,Alea / Tegea,"['Alea', 'Tegea']",276,to 276; b: 276 to 282 \n\n,"['Augusti/Augustae', 'miliaria', 'viri']",miliaria,"['Augusti/Augustae', 'viri']",{},,,to 276; b: 276 to 282 \n\n \n \nImp...,Imp(eratori) / Floriano / P(io) F(elici) Aug(u...,http://db.edcs.eu/epigr/partner.php?s_language...,NaN,276,Augusti/Augustae; miliaria; viri,37.454501,22.420877,

In [9]:
inscr_texts = EDH_sampled["clean_text_interpretive_word"].tolist()

The language of inscriptions is characterized by a number of standardized phrases, which often determine the type of insription. Treated separately, "Dis" and "Manibus" might be confusing, but treated as a bigram, they bear a crucial information about the type of inscription.



In [10]:
def get_bigrams_underscore(inscr_text):
    try:
        inscr_bigrams = [" ".join(el) for el in list(bigrams(inscr_text.split()))]
        inscr_bigrams_ = [bigram.replace(" ", "_") for bigram in inscr_bigrams]
    except:
        inscr_bigrams_ = []
    return inscr_bigrams_

In [11]:
EDH_bigrams_ = [get_bigrams_underscore(inscr_text) for inscr_text in inscr_texts]
EDH_bigrams_[:5]

[['Fortissimo_et',
  'et_piissimo',
  'piissimo_Caesari',
  'Caesari_domino',
  'domino_nostro',
  'nostro_Galerio',
  'Galerio_Valerio',
  'Valerio_Maximiano',
  'Maximiano_Pio',
  'Pio_Felici',
  'Felici_Invicto',
  'Invicto_Coranius',
  'Coranius_Titianus',
  'Titianus_vir',
  'vir_perfectissimus',
  'perfectissimus_praeses',
  'praeses_provinciae',
  'provinciae_veteris',
  'veteris_Epiri',
  'Epiri_numini',
  'numini_eorum',
  'eorum_dicatissimus'],
 ['Imperatori_Caesari',
  'Caesari_Marco',
  'Marco_Annio',
  'Annio_Floriano',
  'Floriano_Pio',
  'Pio_Felici',
  'Felici_Augusto',
  'Augusto_patri',
  'patri_patriae',
  'patriae_milia',
  'milia_passuum',
  'passuum_III',
  'III_Imperatori',
  'Imperatori_Caesari',
  'Caesari_Marco',
  'Marco_Aurelio',
  'Aurelio_Probo',
  'Probo_Pio',
  'Pio_Felici',
  'Felici_Augusto',
  'Augusto_milia',
  'milia_passuum',
  'passuum_II'],
 ['Tiberius_Claudius',
  'Claudius_Caesar',
  'Caesar_Augustus',
  'Augustus_Germanicus',
  'Germanicus_pon

In [12]:
# flat list of all bigrams
bigrams_list = [el for sublist in EDH_bigrams_ for el in sublist]
bigrams_list[:10]

['Fortissimo_et',
 'et_piissimo',
 'piissimo_Caesari',
 'Caesari_domino',
 'domino_nostro',
 'nostro_Galerio',
 'Galerio_Valerio',
 'Valerio_Maximiano',
 'Maximiano_Pio',
 'Pio_Felici']

In [13]:
# ok, let's list only N of the most common
bigrams_mostfreq = nltk.FreqDist(bigrams_list).most_common()
bigrams_mostfreq = [tup[0] for tup in bigrams_mostfreq]
bigrams_mostfreq[:10]

['Dis_Manibus',
 'vixit_annos',
 'votum_solvit',
 'solvit_libens',
 'libens_merito',
 'Iovi_Optimo',
 'Optimo_Maximo',
 'hic_situs',
 'situs_est',
 'tribunicia_potestate']

In [14]:
all_bigrams_freq_dict = dict(nltk.FreqDist(bigrams_list).most_common())

In [15]:
EDH_sampled["bigrams"] = EDH_bigrams_

In [16]:
len(EDH_sampled["type_of_inscription_clean"].unique())

22

In [17]:
all_bigrams_freq_dict

{'Dis_Manibus': 7844,
 'vixit_annos': 4084,
 'votum_solvit': 3335,
 'solvit_libens': 2982,
 'libens_merito': 2710,
 'Iovi_Optimo': 2413,
 'Optimo_Maximo': 2399,
 'hic_situs': 2338,
 'situs_est': 2228,
 'tribunicia_potestate': 1990,
 'Manibus_sacrum': 1865,
 'bene_merenti': 1585,
 'sibi_et': 1476,
 'Imperatori_Caesari': 1256,
 'hic_sita': 1216,
 'sita_est': 1155,
 'faciendum_curavit': 1118,
 'pro_salute': 1117,
 'terra_levis': 1111,
 'Imperator_Caesar': 1099,
 'tibi_terra': 1029,
 'sit_tibi': 999,
 'vixit_annis': 996,
 'pro_praetore': 932,
 'pontifex_maximus': 837,
 'miles_legionis': 783,
 'pontifici_maximo': 780,
 'est_sit': 749,
 'maximus_tribunicia': 708,
 'maximo_tribunicia': 706,
 'ex_voto': 701,
 'Pio_Felici': 698,
 'pater_patriae': 698,
 'patri_patriae': 687,
 'divi_Nervae': 670,
 'et_suis': 664,
 'et_I': 663,
 'Marcus_Aurelius': 656,
 'qui_vixit': 634,
 'piae_fidelis': 611,
 'Caius_Iulius': 608,
 'Marco_Aurelio': 600,
 'libens_laetus': 588,
 'decreto_decurionum': 585,
 'divi_Tra

In [18]:
N = 10
weighted_bigrams_all = []
for cat in EDH_sampled["type_of_inscription_clean"].unique():
    EDH_cat = EDH_sampled[EDH_sampled["type_of_inscription_clean"]==cat]
    bigrams_list = [el for sublist in EDH_cat["bigrams"].tolist() for el in sublist]
    cat_bigrams_freq_dict = dict(nltk.FreqDist(bigrams_list).most_common())
    weighted_bigrams = []
    for word in cat_bigrams_freq_dict.keys():
        if cat_bigrams_freq_dict[word] >= 10: # it it appears at least twice
            weighted_bigrams.append((word, cat_bigrams_freq_dict[word] / all_bigrams_freq_dict[word]))
    # choose only N of the most characteristic bigrams for given category
    weighted_bigrams = sorted(weighted_bigrams, key=lambda tup: tup[1], reverse=True)[:N]
    weighted_bigrams = [tup[0] for tup in weighted_bigrams]
    weighted_bigrams_all.extend(weighted_bigrams)

In [19]:
len(weighted_bigrams_all)

132

In [20]:
def combine_status_list_and_bigrams(el_status, el_bigrams, el_material):
    # preprocess status:
    try: el_status = eval(el_status)
    except: pass
    if isinstance(el_status, list):
        new_el_status = el_status
    elif isinstance(el_status, str):
        new_el_status = [el_status]
    else: 
        new_el_status = []
    new_el_status = [el.replace(" ", "_") for el in new_el_status]
    # preprocess material:
    if el_material is None:
        el_material = []    
    else:
        el_material = el_material.partition(": ")[0] # 
        el_material = el_material.split(", ")
        el_material = [el.replace(" ", "_").replace("?", "") for el in el_material]
    # combine status, bigrams and material
    new_el = new_el_status + el_bigrams + el_material
    new_el = " ".join(new_el)
    return new_el, new_el_status, el_material

In [21]:
status_bigrams_list = []
status_list = []
material_list = []
for el_status, el_bigrams, el_material in zip(EDH_sampled["status_list"].tolist(), EDH_sampled["bigrams"].tolist(), EDH_sampled["Material"].tolist()):
    new_el, new_status, el_material = combine_status_list_and_bigrams(el_status, el_bigrams, el_material)
    status_bigrams_list.append(new_el)
    status_list.append(new_status)
    material_list.extend(el_material)

In [22]:
materials_freqdist = nltk.FreqDist(material_list).most_common()
len(materials_freqdist)

18

In [23]:
materials_freqdist

[('lapis', 22042),
 ('opus_figlinae', 2247),
 ('aes', 796),
 ('plumbum', 528),
 ('lignum', 289),
 ('argentum', 289),
 ('aurum', 82),
 ('vitrum', 63),
 ('ferrum', 60),
 ('tectorium', 58),
 ('musivum', 27),
 ('steatitis', 11),
 ('os', 10),
 ('cyprum', 6),
 ('gemma', 4),
 ('rupes', 4),
 ('corium', 1),
 ('sucineus', 1)]

In [24]:
N = 20
materials_N = [tup[0] for tup in materials_freqdist[:N]]
len(materials_N)

18

In [25]:
status_bigrams_list[:10]

['Augusti/Augustae litterae_erasae ordo_equester tituli_honorarii tria_nomina Fortissimo_et et_piissimo piissimo_Caesari Caesari_domino domino_nostro nostro_Galerio Galerio_Valerio Valerio_Maximiano Maximiano_Pio Pio_Felici Felici_Invicto Invicto_Coranius Coranius_Titianus Titianus_vir vir_perfectissimus perfectissimus_praeses praeses_provinciae provinciae_veteris veteris_Epiri Epiri_numini numini_eorum eorum_dicatissimus',
 'Augusti/Augustae miliaria viri Imperatori_Caesari Caesari_Marco Marco_Annio Annio_Floriano Floriano_Pio Pio_Felici Felici_Augusto Augusto_patri patri_patriae patriae_milia milia_passuum passuum_III III_Imperatori Imperatori_Caesari Caesari_Marco Marco_Aurelio Aurelio_Probo Probo_Pio Pio_Felici Felici_Augusto Augusto_milia milia_passuum passuum_II lapis',
 'Augusti/Augustae leges viri Tiberius_Claudius Claudius_Caesar Caesar_Augustus Augustus_Germanicus Germanicus_pontifex pontifex_maximus maximus_tribunicia tribunicia_potestate potestate_VIIII VIIII_imperator impe

In [26]:
el_status = EDH_sampled["status_list"].tolist()[0]

In [27]:
status_bigrams_list[0]

'Augusti/Augustae litterae_erasae ordo_equester tituli_honorarii tria_nomina Fortissimo_et et_piissimo piissimo_Caesari Caesari_domino domino_nostro nostro_Galerio Galerio_Valerio Valerio_Maximiano Maximiano_Pio Pio_Felici Felici_Invicto Invicto_Coranius Coranius_Titianus Titianus_vir vir_perfectissimus perfectissimus_praeses praeses_provinciae provinciae_veteris veteris_Epiri Epiri_numini numini_eorum eorum_dicatissimus'

In [28]:
status_list_flat = [el for sublist in status_list for el in sublist]
status_N = [tup[0] for tup in nltk.FreqDist(status_list_flat).most_common(100)] ### we had 228 in total
status_N[:10]

['viri',
 'tituli_sepulcrales',
 'tria_nomina',
 'tituli_sacri',
 'mulieres',
 'milites',
 'nomen_singulare',
 'Augusti/Augustae',
 'tituli_operum',
 'tituli_fabricationis']

In [29]:
len(status_N)

37

# Experiments with one list of the most frequent bigrams

In [34]:
vocab = status_N + bigrams_mostfreq[:100] + materials_N

In [33]:
subset_size = 3000 # len(y)
vectorizer = TfidfVectorizer(token_pattern=r"\w+\/?|\_\w+", vocabulary=vocab)
X = vectorizer.fit_transform(status_bigrams_list[:subset_size])
X.todense().shape
X_train, X_test, y_train, y_test = train_test_split(X[:subset_size], y[:subset_size], test_size=0.1, random_state=1)

In [73]:
def train_svm(X, y):
    """
    Create and train the Support Vector Machine.
    """
    svm = SVC(C=1000000.0, gamma='auto', kernel='rbf', probability=True)
    svm.fit(X, y)
    return svm

In [74]:
%%time
svm = train_svm(X_train, y_train)

CPU times: user 2.46 s, sys: 15 ms, total: 2.48 s
Wall time: 2.48 s


In [75]:
print(svm.score(X_test, y_test))

0.8333333333333334


In [38]:
# only where the predictor is present
predictors_present = [np.any(el) for el in X_test.todense()]
# check the performance on datapoints for which we have something in the "status_list"
svm.score(X_test.todense()[predictors_present], np.array(y_test)[predictors_present])

0.8617021276595744

In [39]:
svm.classes_

array(['acclamation', 'assignation inscription', 'boundary inscription',
       'building/dedicatory inscription', 'defixio', 'elogium', 'epitaph',
       'honorific inscription', 'identification inscription', 'label',
       'letter', 'list', 'mile-/leaguestone', 'military diploma',
       'owner/artist inscription', 'prayer', 'private legal inscription',
       'public legal inscription', 'seat inscription',
       'votive inscription'], dtype='<U31')

In [40]:
test_pred_probs = [(svm.classes_[np.argmax(pred)], pred[np.argmax(pred)], real) for pred, real in zip(svm.predict_proba(X_test), y_test)]
test_pred_probs[:20]

[('epitaph', 0.5701242436459757, 'epitaph'),
 ('epitaph', 0.9621257717863589, 'epitaph'),
 ('building/dedicatory inscription', 0.3946471383724565, 'epitaph'),
 ('owner/artist inscription',
  0.6050564118603547,
  'identification inscription'),
 ('votive inscription', 0.9557391731241968, 'votive inscription'),
 ('epitaph', 0.9432540245204908, 'epitaph'),
 ('defixio', 0.4921905918450132, 'defixio'),
 ('votive inscription', 0.9571153789522941, 'votive inscription'),
 ('epitaph', 0.9187064590820762, 'epitaph'),
 ('owner/artist inscription', 0.6617075497938236, 'owner/artist inscription'),
 ('owner/artist inscription', 0.6765519220011015, 'owner/artist inscription'),
 ('votive inscription', 0.6218476701820005, 'votive inscription'),
 ('epitaph', 0.5051037826496995, 'votive inscription'),
 ('epitaph', 0.9231800137716382, 'epitaph'),
 ('owner/artist inscription', 0.6717331047636865, 'owner/artist inscription'),
 ('mile-/leaguestone', 0.8268432464997768, 'mile-/leaguestone'),
 ('epitaph', 0.93

In [41]:
for threshold in [thres / 100 for thres in range(30, 100, 5)]:
    above_threshold = [tup for tup in test_pred_probs if tup[1] >= threshold]
    correct_count = len([tup for tup in above_threshold if tup[0] == tup[2]]) # correct total
    correct_proportion = correct_count / len(above_threshold)# correct ratio 
    proportion = len(above_threshold) / len(test_pred_probs)
    print("prob. threshold: {0}, proportion of inscr.: {1}, correctly classified: {2}".format(threshold, proportion, correct_proportion))

prob. threshold: 0.3, proportion of inscr.: 0.99, correctly classified: 0.8484848484848485
prob. threshold: 0.35, proportion of inscr.: 0.96, correctly classified: 0.8506944444444444
prob. threshold: 0.4, proportion of inscr.: 0.92, correctly classified: 0.8731884057971014
prob. threshold: 0.45, proportion of inscr.: 0.9066666666666666, correctly classified: 0.8786764705882353
prob. threshold: 0.5, proportion of inscr.: 0.8966666666666666, correctly classified: 0.8810408921933085
prob. threshold: 0.55, proportion of inscr.: 0.8566666666666667, correctly classified: 0.8988326848249028
prob. threshold: 0.6, proportion of inscr.: 0.76, correctly classified: 0.9385964912280702
prob. threshold: 0.65, proportion of inscr.: 0.73, correctly classified: 0.9497716894977168
prob. threshold: 0.7, proportion of inscr.: 0.69, correctly classified: 0.9516908212560387
prob. threshold: 0.75, proportion of inscr.: 0.6633333333333333, correctly classified: 0.9547738693467337
prob. threshold: 0.8, proport

# Testing different models

In [45]:
from sklearn import datasets
from sklearn import metrics
from sklearn.model_selection import KFold, cross_val_score
from sklearn.pipeline import make_pipeline 
from sklearn.pipeline import Pipeline 


from sklearn.linear_model import LogisticRegression 
from sklearn.preprocessing import StandardScaler

In [35]:
vectorizer = TfidfVectorizer(token_pattern=r"\w+\/?|\_\w+", vocabulary=vocab)
X = vectorizer.fit_transform(status_bigrams_list)
X.todense().shape

(46171, 155)

In [36]:
classifier = SVC(C=1000000.0, gamma='auto', kernel='rbf', probability=True)
pipeline = make_pipeline(classifier)

In [46]:
pipeline = Pipeline(steps=[("tfidf", vectorizer), ("svm", classifier)]) 

In [49]:
N = 2000 # number of observations
# Create k-Fold cross-validation
kf = KFold(n_splits=10, shuffle=True, random_state=1)

# Conduct k-fold cross-validation 
cv_results = cross_val_score(
    pipeline, # Pipeline
    status_bigrams_list[:N], # Feature matrix
    y[:N], # Target vector 
    cv=kf, # Cross-validation technique 
    scoring="accuracy", # Loss function
    n_jobs=-1) # Use all CPU scores
cv_results

array([0.79 , 0.775, 0.78 , 0.795, 0.795, 0.725, 0.775, 0.755, 0.805,
       0.78 ])

In [37]:
from sklearn.dummy import DummyClassifier
from sklearn.model_selection import train_test_split

In [41]:
X_train, X_test, y_train, y_test = train_test_split(X[:N], y[:N], test_size=0.2, random_state=1)

In [42]:
dummy = DummyClassifier(strategy='stratified', random_state=1)

# "Train" model 
dummy.fit(X_train, y_train)
dummy.score(X_test, y_test)

0.2375

In [50]:
pipeline = Pipeline(steps=[("vectorizer", vectorizer), ("classifier", dummy)]) 

In [51]:
cv_results = cross_val_score(
    dummy, # Pipeline
    X[:N], # Feature matrix
    y[:N], # Target vector 
    cv=kf, # Cross-validation technique 
    scoring="accuracy", # Loss function
    n_jobs=-1) # Use all CPU scores
cv_results

array([0.25 , 0.235, 0.215, 0.205, 0.305, 0.195, 0.2  , 0.225, 0.26 ,
       0.225])

In [57]:
model_variants = [
    {"C" : 1},
    {"C" : 100},
    {"C" : 1000},
    {"C" : 10000},
    {"C" : 100000},
    {"C" : 500000}
    
]

In [ ]:
"gamma" : "linear", 
"gama" : "poly",
"gamma" : "rbf",
"gamma" : "sigmoid",

In [58]:


for var in model_variants:
    svm = SVC(C=var["C"], gamma='auto', kernel='rbf', probability=True)
    svm.fit(X_train, y_train)
    print(svm.score(X_test, y_test))

0.74
0.8633333333333333
0.8666666666666667
0.86
0.8733333333333333
0.86


# ALTERNATIVE: Experiments with the most typical bigrams for each category (skip!)

- quite surprisingly, a model combining status list with 100 most frequent bigrams outperforms a model based on 10 most typical bigrams within each category

In [66]:
subset_size = 3000 # len(y)
vocab = status_N + weighted_bigrams_all
vectorizer = TfidfVectorizer(token_pattern=r"\w+\/?|\_\w+", vocabulary=vocab)
X = vectorizer.fit_transform(status_bigrams_list[:subset_size])
X.todense().shape
X_train, X_test, y_train, y_test = train_test_split(X[:subset_size], y[:subset_size], test_size=0.1, random_state=1)

In [67]:
%%time
svm = train_svm(X_train, y_train)

CPU times: user 3.82 s, sys: 0 ns, total: 3.82 s
Wall time: 3.82 s


In [68]:
print(svm.score(X_test, y_test))

0.8166666666666667


In [69]:
# only where the predictor is present
predictors_present = [np.any(el) for el in X_test.todense()]
# check the performance on datapoints for which we have something in the "status_list"
svm.score(X_test.todense()[predictors_present], np.array(y_test)[predictors_present])

0.8935361216730038

#  Main training

In [293]:
vocab = status_N + bigrams_mostfreq[:100] + materials_N
vectorizer = TfidfVectorizer(token_pattern=r"\w+\/?|\_\w+", vocabulary=vocab)
X = vectorizer.fit_transform(status_bigrams_list)
X.todense().shape
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.1, random_state=1)

In [294]:
%%time
svm = train_svm(X_train, y_train)

CPU times: user 4h 26min 11s, sys: 146 ms, total: 4h 26min 11s
Wall time: 4h 26min 17s


In [295]:
print(svm.score(X_test, y_test))

0.8607622347336509


In [296]:
# only where the predictor is present
predictors_present = [np.any(el) for el in X_test.todense()]
# check the performance on datapoints for which we have something in the "status_list"
svm.score(X_test.todense()[predictors_present], np.array(y_test)[predictors_present])

0.8873664653971203

In [333]:
test_pred_probs = [(svm.classes_[np.argmax(pred)], pred[np.argmax(pred)], real) for pred, real in zip(svm.predict_proba(X_test), y_test)]
test_pred_probs[:20]

[('epitaph', 0.8076518568979559, 'epitaph'),
 ('epitaph', 0.9175135155645008, 'epitaph'),
 ('honorific inscription', 0.8175375756362547, 'epitaph'),
 ('votive inscription', 0.9375336045590539, 'votive inscription'),
 ('epitaph', 0.9889070296976127, 'epitaph'),
 ('honorific inscription', 0.4244074964132119, 'honorific inscription'),
 ('identification inscription', 0.38499247315960017, 'acclamation'),
 ('epitaph', 0.6814848074358177, 'epitaph'),
 ('votive inscription', 0.8981249140959828, 'votive inscription'),
 ('epitaph', 0.9898373617328683, 'epitaph'),
 ('epitaph', 0.9995455498440634, 'epitaph'),
 ('owner/artist inscription', 0.7672183828711583, 'owner/artist inscription'),
 ('building/dedicatory inscription', 0.3477246765155101, 'epitaph'),
 ('building/dedicatory inscription',
  0.5086770937852645,
  'building/dedicatory inscription'),
 ('votive inscription', 0.8722865158161045, 'votive inscription'),
 ('votive inscription', 0.9771523245027367, 'votive inscription'),
 ('votive inscri

In [336]:
test_results = []
for threshold in [thres / 100 for thres in range(30, 100, 5)]:
    above_threshold = [tup for tup in test_pred_probs if tup[1] >= threshold]
    correct_count = len([tup for tup in above_threshold if tup[0] == tup[2]]) # correct total
    correct_proportion = correct_count / len(above_threshold)# correct ratio 
    proportion = len(above_threshold) / len(test_pred_probs)
    test_results.append({"threshold (=/>)" : threshold, "proportion" : np.round(proportion, 2), "N" : len(above_threshold), "correct" : correct_proportion})
test_results_df = pd.DataFrame(test_results)
test_results_df

,threshold (=/>),proportion,N,correct
0,0.30,1.00,4601,0.862421
1,0.35,0.94,4337,0.890247
2,0.40,0.93,4310,0.894200
3,0.45,0.91,4213,0.898647
4,0.50,0.90,4162,0.901970
5,0.55,0.89,4092,0.905670
6,0.60,0.86,3955,0.908976
7,0.65,0.84,3898,0.910723
8,0.70,0.71,3264,0.957414
9,0.75,0.67,3078,0.968811


In [338]:
test_results_df.to_csv("../data/svm_v1_test_results.csv")

In [307]:
model_path = '../data/svm_model_v1.0.sav'
pickle.dump(svm, open(model_path, 'wb'))
vectorizer_path= '../data/svm_vectorizer_v1.0.sav'
pickle.dump(vectorizer, open(vectorizer_path, "wb"))

# Applying the model

In [310]:
# load the model and vectorizer back
svm_v1 = pickle.load(open('../data/svm_model_v1.0.sav', 'rb'))
vectorizer_v1 = pickle.load(open('../data/svm_vectorizer_v1.0.sav','rb'))

In [311]:
# we will apply the model to inscriptions which are only in EDCS
EDCS_unique = EDHCSg[(EDHCSg["EDH-ID"].isnull()) & (EDHCSg["EDCS-ID"].notnull())] 
EDCS_unique.head(5)

,EDCS-ID,publication,province,province_list,place,place_list,end_yr_list,notes_dating,status_list,inscr_type,status_notation,inscr_process,notes_references,notes_comments,inscription,inscription_stripped_final,Links,dating from,dating to,status,Latitude,Longitude,photo,Material,Comment,EDH-ID,language_EDCS,clean_text_interpretive_word_EDCS,responsible_individual,type_of_inscription,letter_size,literature,work_status,height,diplomatic_text,people,depth,material,type_of_monument,province_label,width,transcription,country,uri,findspot_ancient,last_update,modern_region,findspot_modern,edh_geography_uri,commentary,trismegistos_uri,external_image_uris,fotos,idno_tm,placenames_refs,text_edition,origdate_text,layout_execution,layout_execution_text,support_objecttype,support_objecttype_text,support_material,support_material_text,support_decoration,keywords_term,keywords_term_text,type_of_inscription_clean,type_of_inscription_certainty,height_cm,width_cm,depth_cm,material_clean,type_of_monument_clean,type_of_monument_certainty,province_label_clean,province_label_certainty,country_clean,country_certainty,findspot_ancient_clean,findspot_ancient_certainty,modern_region_clean,modern_region_certainty,findspot_modern_clean,findspot_modern_certainty,findspot_clean,findspot_certainty,origdate_text_clean,clean_text_conservative,clean_text_interpretive_sentence,findspot,year_of_find,present_location,religion,geography,social_economic_legal_history,military,not_after,language,not_before,coordinates,clean_text_interpretive_word,urban_context,within_rome,nearest_city,city_id_hanson,city_pop_est,city_geometry,nearest_city_type,nearest_city_dist,geometry,bigrams
0,EDCS-71300269,"AE 2014, 01203",Achaia,Achaia,Mavrommati / Mavromati / Messini / Messene,"['Mavrommati', 'Mavromati', 'Messini', 'Messene']",96,,"['sigilla impressa', 'tituli fabricationis', '...",tituli fabricationis,viri,sigilla impressa,,,De Rasticanis,De Rasticanis,None,69.0,96,sigilla impressa; tituli fabricationis; viri,37.179412,21.924179,None,opus figlinae,None,None,None,De Rasticanis,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,NaN,None,None,None,NaN,None,None,None,None,NaN,NaN,NaN,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,96.0,Latin,69.0,"['21.9241791', '37.1794122']",De Rasticanis,big,False,Messene,59,22337.0,"[22.008029, 37.051363]",middle,0.153060,POINT (21.92418 37.17941),[De_Rasticanis]
1,EDCS-21200140,"CIL 03, 00079 = CIL 03, 14147,6 = IDakke 00028",Aegyptus,Aegyptus,Ad-Dakka / El-Dakka / Dakke / Pselqet / Pselki...,"['Ad-Dakka', 'El-Dakka', 'Dakke', 'Pselqet', '...",109,,"['milites', 'tituli fabricationis', 'tituli sa...","['tituli fabricationis', 'tituli sacri']","['milites', 'tria nomina', 'viri']",{},,,Deo Magno Mercurio / adoravit vexillu / leg(io...,Deo Magno Mercurio / adoravit vexillu / leg(io...,http://db.edcs.eu/epigr/partner.php?s_language...,109.0,109,milites; tituli fabricationis; tituli sacri; t...,23.200000,32.750000,None,None,None,None,None,Deo Magno Mercurio adoravit vexillu legionis I...,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,NaN,None,None,None,NaN,None,None,None,None,NaN,NaN,NaN,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,109.0,Latin,109.0,"['32.750000', '23.200000']",Deo Magno Mercurio adoravit vexillu legionis I...,rest,False,Elephantine,126,33806.0,"[32.933333, 24.083333]",big,0.902158,POINT (32.75000 23.20000),"[Deo_Magno, Magno_Mercurio, Mercurio_adoravit,..."
2,EDCS-21200142,"CIL 03, 00081 = CIL 03, 13584 = IDakke 00064a",Aegyptus,Aegyptus,Ad-Dakka / El-Dakka / Dakke / Pselqet / Pselki...,"['Ad-Dakka', 'El-Dakka', 'Dakke', 'Pselqet', '...",109,,{},{},{},{},,,Anno / XII 

In [312]:
# first create bigrams
EDCS_unique["bigrams"] = EDCS_unique["clean_text_interpretive_word"].apply(get_bigrams_underscore)

In [313]:
# we need to extract all predictors from the dataframe
# into `status_bigrams_list_apply` (list of lists)
status_bigrams_list_apply = []
status_list = []
material_list = []
for el_status, el_bigrams, el_material in zip(EDCS_unique["status_list"].tolist(), EDCS_unique["bigrams"].tolist(), EDCS_unique["Material"].tolist()):
    new_el, new_status, el_material = combine_status_list_and_bigrams(el_status, el_bigrams, el_material)
    status_bigrams_list_apply.append(new_el)
    status_list.append(new_status)
    material_list.extend(el_material)

In [315]:
X_applied = vectorizer_v1.transform(status_bigrams_list_apply)

In [316]:
prediction = svm_v1.predict(X_applied)
prediction[:10]

array(['owner/artist inscription', 'owner/artist inscription', 'epitaph',
       'votive inscription', 'votive inscription', 'epitaph', 'epitaph',
       'honorific inscription', 'honorific inscription',
       'honorific inscription'], dtype='<U31')

In [377]:
svm_v1.classes_

array(['acclamation', 'adnuntiatio', 'assignation inscription',
       'boundary inscription', 'building/dedicatory inscription',
       'calendar', 'defixio', 'elogium', 'epitaph',
       'honorific inscription', 'identification inscription', 'label',
       'letter', 'list', 'mile-/leaguestone', 'military diploma',
       'owner/artist inscription', 'prayer', 'private legal inscription',
       'public legal inscription', 'seat inscription',
       'votive inscription'], dtype='<U31')

In [317]:
probs = [np.max(pred) for pred in svm_v1.predict_proba(X_applied)]
probs[:10]

[0.7857018706636444,
 0.850080323385903,
 0.696566811086934,
 0.7255255264157866,
 0.91963276157691,
 0.9973812084603909,
 0.722058041187063,
 0.5854847852357029,
 0.8361305582932681,
 0.9281131587280677]

In [318]:
prediction_probs = [(svm_v1.classes_[np.argmax(pred)], pred[np.argmax(pred)]) for pred in svm_v1.predict_proba(X_applied)]
prediction_probs[:10]

[('owner/artist inscription', 0.7857018706636444),
 ('owner/artist inscription', 0.850080323385903),
 ('epitaph', 0.696566811086934),
 ('votive inscription', 0.7255255264157866),
 ('votive inscription', 0.91963276157691),
 ('epitaph', 0.9973812084603909),
 ('epitaph', 0.722058041187063),
 ('honorific inscription', 0.5854847852357029),
 ('honorific inscription', 0.8361305582932681),
 ('honorific inscription', 0.9281131587280677)]

In [320]:
len(prediction_probs)

83482

In [321]:
for threshold in [thres / 100 for thres in range(30, 100, 5)]:
    above_threshold = [tup for tup in prediction_probs if tup[1] >= threshold]
    proportion = len(above_threshold) / len(prediction_probs)
    print("prob. threshold: {0}, proportion of inscr.: {1}".format(threshold, proportion))

prob. threshold: 0.3, proportion of inscr.: 0.9917227665844134
prob. threshold: 0.35, proportion of inscr.: 0.9776718334491268
prob. threshold: 0.4, proportion of inscr.: 0.9708440142785271
prob. threshold: 0.45, proportion of inscr.: 0.9533192784073213
prob. threshold: 0.5, proportion of inscr.: 0.9454612970460698
prob. threshold: 0.55, proportion of inscr.: 0.9377949737668
prob. threshold: 0.6, proportion of inscr.: 0.9269543135047076
prob. threshold: 0.65, proportion of inscr.: 0.920713447210177
prob. threshold: 0.7, proportion of inscr.: 0.732073980019645
prob. threshold: 0.75, proportion of inscr.: 0.7075776814163532
prob. threshold: 0.8, proportion of inscr.: 0.605962962075657
prob. threshold: 0.85, proportion of inscr.: 0.5564073692532522
prob. threshold: 0.9, proportion of inscr.: 0.4863443616588007
prob. threshold: 0.95, proportion of inscr.: 0.4326441628135406


In [328]:
threshold_results = []
for threshold in [thres / 100 for thres in range(30, 100, 5)]:
    above_threshold = [tup for tup in prediction_probs if tup[1] >= threshold]
    proportion = len(above_threshold) / len(prediction_probs)
    threshold_results.append({"threshold (=/>)" : threshold, "proportion" : np.round(proportion, 2), "N" : len(above_threshold)})
model_results_df = pd.DataFrame(threshold_results)
model_results_df

,threshold (=/>),proportion,N
0,0.30,0.99,82791
1,0.35,0.98,81618
2,0.40,0.97,81048
3,0.45,0.95,79585
4,0.50,0.95,78929
5,0.55,0.94,78289
6,0.60,0.93,77384
7,0.65,0.92,76863
8,0.70,0.73,61115
9,0.75,0.71,59070


In [332]:
model_results_df.to_csv("../data/svm_v1_EDCS_results.csv")

In [337]:
test_results_df

,threshold (=/>),proportion,N,correct
0,0.30,1.00,4601,0.862421
1,0.35,0.94,4337,0.890247
2,0.40,0.93,4310,0.894200
3,0.45,0.91,4213,0.898647
4,0.50,0.90,4162,0.901970
5,0.55,0.89,4092,0.905670
6,0.60,0.86,3955,0.908976
7,0.65,0.84,3898,0.910723
8,0.70,0.71,3264,0.957414
9,0.75,0.67,3078,0.968811


# Combining the data back

In [339]:
EDCS_unique["type_of_inscription_auto"] = prediction
EDCS_unique["type_of_inscription_auto_prob"] = probs

In [340]:
EDCS_unique.head(5)

,EDCS-ID,publication,province,province_list,place,place_list,end_yr_list,notes_dating,status_list,inscr_type,status_notation,inscr_process,notes_references,notes_comments,inscription,inscription_stripped_final,Links,dating from,dating to,status,Latitude,Longitude,photo,Material,Comment,EDH-ID,language_EDCS,clean_text_interpretive_word_EDCS,responsible_individual,type_of_inscription,letter_size,literature,work_status,height,diplomatic_text,people,depth,material,type_of_monument,province_label,width,transcription,country,uri,findspot_ancient,last_update,modern_region,findspot_modern,edh_geography_uri,commentary,trismegistos_uri,external_image_uris,fotos,idno_tm,placenames_refs,text_edition,origdate_text,layout_execution,layout_execution_text,support_objecttype,support_objecttype_text,support_material,support_material_text,support_decoration,keywords_term,keywords_term_text,type_of_inscription_clean,type_of_inscription_certainty,height_cm,width_cm,depth_cm,material_clean,type_of_monument_clean,type_of_monument_certainty,province_label_clean,province_label_certainty,country_clean,country_certainty,findspot_ancient_clean,findspot_ancient_certainty,modern_region_clean,modern_region_certainty,findspot_modern_clean,findspot_modern_certainty,findspot_clean,findspot_certainty,origdate_text_clean,clean_text_conservative,clean_text_interpretive_sentence,findspot,year_of_find,present_location,religion,geography,social_economic_legal_history,military,not_after,language,not_before,coordinates,clean_text_interpretive_word,urban_context,within_rome,nearest_city,city_id_hanson,city_pop_est,city_geometry,nearest_city_type,nearest_city_dist,geometry,bigrams,type_of_inscription_auto,type_of_inscription_auto_prob
0,EDCS-71300269,"AE 2014, 01203",Achaia,Achaia,Mavrommati / Mavromati / Messini / Messene,"['Mavrommati', 'Mavromati', 'Messini', 'Messene']",96,,"['sigilla impressa', 'tituli fabricationis', '...",tituli fabricationis,viri,sigilla impressa,,,De Rasticanis,De Rasticanis,None,69.0,96,sigilla impressa; tituli fabricationis; viri,37.179412,21.924179,None,opus figlinae,None,None,None,De Rasticanis,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,NaN,None,None,None,NaN,None,None,None,None,NaN,NaN,NaN,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,96.0,Latin,69.0,"['21.9241791', '37.1794122']",De Rasticanis,big,False,Messene,59,22337.0,"[22.008029, 37.051363]",middle,0.153060,POINT (21.92418 37.17941),[De_Rasticanis],owner/artist inscription,0.785702
1,EDCS-21200140,"CIL 03, 00079 = CIL 03, 14147,6 = IDakke 00028",Aegyptus,Aegyptus,Ad-Dakka / El-Dakka / Dakke / Pselqet / Pselki...,"['Ad-Dakka', 'El-Dakka', 'Dakke', 'Pselqet', '...",109,,"['milites', 'tituli fabricationis', 'tituli sa...","['tituli fabricationis', 'tituli sacri']","['milites', 'tria nomina', 'viri']",{},,,Deo Magno Mercurio / adoravit vexillu / leg(io...,Deo Magno Mercurio / adoravit vexillu / leg(io...,http://db.edcs.eu/epigr/partner.php?s_language...,109.0,109,milites; tituli fabricationis; tituli sacri; t...,23.200000,32.750000,None,None,None,None,None,Deo Magno Mercurio adoravit vexillu legionis I...,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,NaN,None,None,None,NaN,None,None,None,None,NaN,NaN,NaN,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,109.0,Latin,109.0,"['32.750000', '23.200000']",Deo Magno Mercurio adoravit vexillu legionis I...,rest,False,Elephantine,126,33806.0,"[32.933333, 24.083333]",big,0.902158,POINT (32.75000 23.20000),"[Deo_Magno, Magno_Mercurio, Mercurio_adoravit,...",owner/artist inscription,0.850080
2,EDCS-21200142,"CIL 03, 00081 = CIL 03, 13584 = IDakke 00064a",Aegyptus,Aegyptus,Ad-Dakka 

In [341]:
# prepare last part of the data - inscriptions we have only in EDH
# in this case, we can also use the information we already have
EDH_unique = EDHCSg[(EDHCSg["EDH-ID"].notnull()) & (EDHCSg["EDCS-ID"].isnull())] 
EDH_unique["type_of_inscription_auto"] = EDH_unique["type_of_inscription_clean"]
EDH_unique["type_of_inscription_auto_prob"] = [1] * len(EDH_unique)

In [342]:
EDH_overlap_all["type_of_inscription_auto"] = EDH_overlap_all["type_of_inscription_clean"]
EDH_overlap_all["type_of_inscription_auto_prob"] = [1] * len(EDH_overlap_all)

In [343]:
print("the dataset we used for training and testing (+NULLs): ", EDH_overlap_all.shape)
print("inscriptions in EDH only: ", EDH_unique.shape)
print("inscriptions in EDCS only, we wanted to classify: ", EDCS_unique.shape)

the dataset we used for training and testing (+NULLs):  (49916, 113)
inscriptions in EDH only:  (3907, 113)
inscriptions in EDCS only, we wanted to classify:  (83482, 113)


In [344]:
# combine together
EDHCS_autotypes = pd.concat([EDH_overlap_all, EDH_unique, EDCS_unique])
EDHCS_autotypes.head()

,EDCS-ID,publication,province,province_list,place,place_list,end_yr_list,notes_dating,status_list,inscr_type,status_notation,inscr_process,notes_references,notes_comments,inscription,inscription_stripped_final,Links,dating from,dating to,status,Latitude,Longitude,photo,Material,Comment,EDH-ID,language_EDCS,clean_text_interpretive_word_EDCS,responsible_individual,type_of_inscription,letter_size,literature,work_status,height,diplomatic_text,people,depth,material,type_of_monument,province_label,width,transcription,country,uri,findspot_ancient,last_update,modern_region,findspot_modern,edh_geography_uri,commentary,trismegistos_uri,external_image_uris,fotos,idno_tm,placenames_refs,text_edition,origdate_text,layout_execution,layout_execution_text,support_objecttype,support_objecttype_text,support_material,support_material_text,support_decoration,keywords_term,keywords_term_text,type_of_inscription_clean,type_of_inscription_certainty,height_cm,width_cm,depth_cm,material_clean,type_of_monument_clean,type_of_monument_certainty,province_label_clean,province_label_certainty,country_clean,country_certainty,findspot_ancient_clean,findspot_ancient_certainty,modern_region_clean,modern_region_certainty,findspot_modern_clean,findspot_modern_certainty,findspot_clean,findspot_certainty,origdate_text_clean,clean_text_conservative,clean_text_interpretive_sentence,findspot,year_of_find,present_location,religion,geography,social_economic_legal_history,military,not_after,language,not_before,coordinates,clean_text_interpretive_word,urban_context,within_rome,nearest_city,city_id_hanson,city_pop_est,city_geometry,nearest_city_type,nearest_city_dist,geometry,bigrams,type_of_inscription_auto,type_of_inscription_auto_prob
83482,EDCS-03700724,"ZPE-108-159 = Thesprotia 00001 = AE 1993, 0140...",Achaia,Achaia,Agios Athanasios / Photike,"['Agios Athanasios', 'Photike']",313,,"['Augusti/Augustae', 'litterae erasae', 'ordo ...",tituli honorarii,"['Augusti/Augustae', 'ordo equester', 'tria no...",litterae erasae,,,Fortissimo et Piis/simo Caesari d(omino) n(ost...,Fortissimo et Piis/simo Caesari d(omino) n(ost...,http://db.edcs.eu/epigr/partner.php?s_language...,309.0,313,Augusti/Augustae; litterae erasae; ordo equest...,39.451218,20.766767,http://db.edcs.eu/epigr/bilder.php?bilder.php?...,None,None,HD052964,None,Fortissimo et Piissimo Caesari domino nostro G...,Cowey,honorific inscription,3-5.3 cm,"AE 1993, 1406.; V. Papadopoulou, AD 43 B, 1988...",checked with photo,99 cm,FORTISSIMO ET PIIS / SIMO CAESARI D N / GAL VA...,"[{'name': 'Gal. Val. [[Maximiano]]', 'nomen': ...",67 cm,None,statue base,Epirus,67 cm,Fortissimo et piis/simo Caesari d(omino) n(ost...,Greece,https://edh-www.adw.uni-heidelberg.de/edh/insc...,Photike,2014-09-16,Ípeiros,Paramythía,https://edh-www.adw.uni-heidelberg.de/edh/geog...,None,https://www.trismegistos.org/text/121715,{},{},121715,"['http://www.trismegistos.org/place/000655', '...",Fortissimo et piissimo Caesari...,309 AD – 313 AD,21,unbestimmt,57.0,Statuenbasis,138,unbestimmt,1000.0,69,Ehreninschrift,honorific inscription,Certain,99.0,67.0,67.0,NULL,statue base,Certain,Epirus,Certain,Greece,Certain,Photike,Certain,Ípeiros,Certain,Paramythía,Certain,Agios Athanasios,Certain,309 AD – 313 AD,Fortissimo et piissimo Caesari d n Gal Val P F...,Fortissimo et piissimo Caesari domino nostro G...,Agios Athanasios,None,Fragma Kalama,None,data available,None,None,313.0,Latin,309.0,"[20.7668, 39.4512]",Fortissimo et piissimo Caesari domino nostro G...,rest,False,Dodona,31,1000.0,"[20.787767, 39.546432]",minor,0.097513,POINT (20.76680 39.45120),"[Fortissimo_et, et_piissimo, piissimo_Caesari,...",honorific inscription,1.0
83483,EDCS-03300852,"AE 1995, 01409",Achaia,Achaia,Alea / Tegea,"['Alea', 'Tegea']",276,to 276; b: 276 to 282 \n\n,"['Augusti/Augustae', 'miliaria', 'viri']",miliaria,"['Augusti/Augustae', 'viri']",{},,,to 276; b: 276 to 282 \n\n \n \nImp...,Imp(eratori) / Floriano / P(io) F(elici) Aug(u...,http://db.edcs.eu/epigr/partner.php?s_langu

In [345]:
EDHCS_autotypes.drop(["bigrams"], axis=1, inplace=True)

In [346]:
EDHCS_autotypes.shape

(137305, 112)

In [347]:
EDHCS_autotypes.head(5)

,EDCS-ID,publication,province,province_list,place,place_list,end_yr_list,notes_dating,status_list,inscr_type,status_notation,inscr_process,notes_references,notes_comments,inscription,inscription_stripped_final,Links,dating from,dating to,status,Latitude,Longitude,photo,Material,Comment,EDH-ID,language_EDCS,clean_text_interpretive_word_EDCS,responsible_individual,type_of_inscription,letter_size,literature,work_status,height,diplomatic_text,people,depth,material,type_of_monument,province_label,width,transcription,country,uri,findspot_ancient,last_update,modern_region,findspot_modern,edh_geography_uri,commentary,trismegistos_uri,external_image_uris,fotos,idno_tm,placenames_refs,text_edition,origdate_text,layout_execution,layout_execution_text,support_objecttype,support_objecttype_text,support_material,support_material_text,support_decoration,keywords_term,keywords_term_text,type_of_inscription_clean,type_of_inscription_certainty,height_cm,width_cm,depth_cm,material_clean,type_of_monument_clean,type_of_monument_certainty,province_label_clean,province_label_certainty,country_clean,country_certainty,findspot_ancient_clean,findspot_ancient_certainty,modern_region_clean,modern_region_certainty,findspot_modern_clean,findspot_modern_certainty,findspot_clean,findspot_certainty,origdate_text_clean,clean_text_conservative,clean_text_interpretive_sentence,findspot,year_of_find,present_location,religion,geography,social_economic_legal_history,military,not_after,language,not_before,coordinates,clean_text_interpretive_word,urban_context,within_rome,nearest_city,city_id_hanson,city_pop_est,city_geometry,nearest_city_type,nearest_city_dist,geometry,type_of_inscription_auto,type_of_inscription_auto_prob
83482,EDCS-03700724,"ZPE-108-159 = Thesprotia 00001 = AE 1993, 0140...",Achaia,Achaia,Agios Athanasios / Photike,"['Agios Athanasios', 'Photike']",313,,"['Augusti/Augustae', 'litterae erasae', 'ordo ...",tituli honorarii,"['Augusti/Augustae', 'ordo equester', 'tria no...",litterae erasae,,,Fortissimo et Piis/simo Caesari d(omino) n(ost...,Fortissimo et Piis/simo Caesari d(omino) n(ost...,http://db.edcs.eu/epigr/partner.php?s_language...,309.0,313,Augusti/Augustae; litterae erasae; ordo equest...,39.451218,20.766767,http://db.edcs.eu/epigr/bilder.php?bilder.php?...,None,None,HD052964,None,Fortissimo et Piissimo Caesari domino nostro G...,Cowey,honorific inscription,3-5.3 cm,"AE 1993, 1406.; V. Papadopoulou, AD 43 B, 1988...",checked with photo,99 cm,FORTISSIMO ET PIIS / SIMO CAESARI D N / GAL VA...,"[{'name': 'Gal. Val. [[Maximiano]]', 'nomen': ...",67 cm,None,statue base,Epirus,67 cm,Fortissimo et piis/simo Caesari d(omino) n(ost...,Greece,https://edh-www.adw.uni-heidelberg.de/edh/insc...,Photike,2014-09-16,Ípeiros,Paramythía,https://edh-www.adw.uni-heidelberg.de/edh/geog...,None,https://www.trismegistos.org/text/121715,{},{},121715,"['http://www.trismegistos.org/place/000655', '...",Fortissimo et piissimo Caesari...,309 AD – 313 AD,21,unbestimmt,57.0,Statuenbasis,138,unbestimmt,1000.0,69,Ehreninschrift,honorific inscription,Certain,99.0,67.0,67.0,NULL,statue base,Certain,Epirus,Certain,Greece,Certain,Photike,Certain,Ípeiros,Certain,Paramythía,Certain,Agios Athanasios,Certain,309 AD – 313 AD,Fortissimo et piissimo Caesari d n Gal Val P F...,Fortissimo et piissimo Caesari domino nostro G...,Agios Athanasios,None,Fragma Kalama,None,data available,None,None,313.0,Latin,309.0,"[20.7668, 39.4512]",Fortissimo et piissimo Caesari domino nostro G...,rest,False,Dodona,31,1000.0,"[20.787767, 39.546432]",minor,0.097513,POINT (20.76680 39.45120),honorific inscription,1.0
83483,EDCS-03300852,"AE 1995, 01409",Achaia,Achaia,Alea / Tegea,"['Alea', 'Tegea']",276,to 276; b: 276 to 282 \n\n,"['Augusti/Augustae', 'miliaria', 'viri']",miliaria,"['Augusti/Augustae', 'viri']",{},,,to 276; b: 276 to 282 \n\n \n \nImp...,Imp(eratori) / Floriano / P(io) F(elici) Aug(u...,http://db.edcs.eu/epigr/partner.php?s_language...,NaN,276,Augusti/Augustae; miliaria; viri,37.454501,22

In [348]:
len(EDHCS_autotypes[EDHCS_autotypes["type_of_inscription_auto_prob"]>0.55])

132112

In [349]:
len(EDHCS_autotypes[EDHCS_autotypes["type_of_inscription_auto"]=="NULL"])

4339

In [350]:
def apply_threshold(inscr_type, prob):
    if (prob < 0.55) or (inscr_type == "NULL"):
        return None 
    else:
        return inscr_type
EDHCS_autotypes["type_of_inscription_auto"] =  EDHCS_autotypes.apply(lambda row: apply_threshold(row["type_of_inscription_auto"], row["type_of_inscription_auto_prob"]), axis=1)

In [365]:
def prob_to_zero(inscr_type, prob):
    if inscr_type == None:
        return 0
    else:
        return prob
EDHCS_autotypes["type_of_inscription_auto_prob"] =  EDHCS_autotypes.apply(lambda row: prob_to_zero(row["type_of_inscription_auto"], row["type_of_inscription_auto_prob"]), axis=1)

In [366]:
len(EDHCS_autotypes[EDHCS_autotypes["type_of_inscription_auto"].notnull()])

127773

In [367]:
len(EDHCS_autotypes[(EDHCS_autotypes["type_of_inscription_auto"].notnull()) & (EDHCS_autotypes["type_of_inscription_auto_prob"] > 0.55)])

127773

In [368]:
s = sddk.cloudSession("sciencedata.dk", "SDAM_root/SDAM_data/EDHCS", "648597@au.dk")

Your ScienceData username (e.g. '123456@au.dk'): 648597@au.dk
Your ScienceData password: ········
connection with shared folder established with you as its owner
endpoint variable has been configured to: https://sciencedata.dk/files/SDAM_root/SDAM_data/EDHCS/


In [371]:
EDHCS_autotypes.head(5)

,EDCS-ID,publication,province,province_list,place,place_list,end_yr_list,notes_dating,status_list,inscr_type,status_notation,inscr_process,notes_references,notes_comments,inscription,inscription_stripped_final,Links,dating from,dating to,status,Latitude,Longitude,photo,Material,Comment,EDH-ID,language_EDCS,clean_text_interpretive_word_EDCS,responsible_individual,type_of_inscription,letter_size,literature,work_status,height,diplomatic_text,people,depth,material,type_of_monument,province_label,width,transcription,country,uri,findspot_ancient,last_update,modern_region,findspot_modern,edh_geography_uri,commentary,trismegistos_uri,external_image_uris,fotos,idno_tm,placenames_refs,text_edition,origdate_text,layout_execution,layout_execution_text,support_objecttype,support_objecttype_text,support_material,support_material_text,support_decoration,keywords_term,keywords_term_text,type_of_inscription_clean,type_of_inscription_certainty,height_cm,width_cm,depth_cm,material_clean,type_of_monument_clean,type_of_monument_certainty,province_label_clean,province_label_certainty,country_clean,country_certainty,findspot_ancient_clean,findspot_ancient_certainty,modern_region_clean,modern_region_certainty,findspot_modern_clean,findspot_modern_certainty,findspot_clean,findspot_certainty,origdate_text_clean,clean_text_conservative,clean_text_interpretive_sentence,findspot,year_of_find,present_location,religion,geography,social_economic_legal_history,military,not_after,language,not_before,coordinates,clean_text_interpretive_word,urban_context,within_rome,nearest_city,city_id_hanson,city_pop_est,city_geometry,nearest_city_type,nearest_city_dist,geometry,type_of_inscription_auto,type_of_inscription_auto_prob
83482,EDCS-03700724,"ZPE-108-159 = Thesprotia 00001 = AE 1993, 0140...",Achaia,Achaia,Agios Athanasios / Photike,"['Agios Athanasios', 'Photike']",313,,"['Augusti/Augustae', 'litterae erasae', 'ordo ...",tituli honorarii,"['Augusti/Augustae', 'ordo equester', 'tria no...",litterae erasae,,,Fortissimo et Piis/simo Caesari d(omino) n(ost...,Fortissimo et Piis/simo Caesari d(omino) n(ost...,http://db.edcs.eu/epigr/partner.php?s_language...,309.0,313,Augusti/Augustae; litterae erasae; ordo equest...,39.451218,20.766767,http://db.edcs.eu/epigr/bilder.php?bilder.php?...,None,None,HD052964,None,Fortissimo et Piissimo Caesari domino nostro G...,Cowey,honorific inscription,3-5.3 cm,"AE 1993, 1406.; V. Papadopoulou, AD 43 B, 1988...",checked with photo,99 cm,FORTISSIMO ET PIIS / SIMO CAESARI D N / GAL VA...,"[{'name': 'Gal. Val. [[Maximiano]]', 'nomen': ...",67 cm,None,statue base,Epirus,67 cm,Fortissimo et piis/simo Caesari d(omino) n(ost...,Greece,https://edh-www.adw.uni-heidelberg.de/edh/insc...,Photike,2014-09-16,Ípeiros,Paramythía,https://edh-www.adw.uni-heidelberg.de/edh/geog...,None,https://www.trismegistos.org/text/121715,{},{},121715,"['http://www.trismegistos.org/place/000655', '...",Fortissimo et piissimo Caesari...,309 AD – 313 AD,21,unbestimmt,57.0,Statuenbasis,138,unbestimmt,1000.0,69,Ehreninschrift,honorific inscription,Certain,99.0,67.0,67.0,NULL,statue base,Certain,Epirus,Certain,Greece,Certain,Photike,Certain,Ípeiros,Certain,Paramythía,Certain,Agios Athanasios,Certain,309 AD – 313 AD,Fortissimo et piissimo Caesari d n Gal Val P F...,Fortissimo et piissimo Caesari domino nostro G...,Agios Athanasios,None,Fragma Kalama,None,data available,None,None,313.0,Latin,309.0,"[20.7668, 39.4512]",Fortissimo et piissimo Caesari domino nostro G...,rest,False,Dodona,31,1000.0,"[20.787767, 39.546432]",minor,0.097513,POINT (20.76680 39.45120),honorific inscription,1.0
83483,EDCS-03300852,"AE 1995, 01409",Achaia,Achaia,Alea / Tegea,"['Alea', 'Tegea']",276,to 276; b: 276 to 282 \n\n,"['Augusti/Augustae', 'miliaria', 'viri']",miliaria,"['Augusti/Augustae', 'viri']",{},,,to 276; b: 276 to 282 \n\n \n \nImp...,Imp(eratori) / Floriano / P(io) F(elici) Aug(u...,http://db.edcs.eu/epigr/partner.php?s_language...,NaN,276,Augusti/Augustae; miliaria; viri,37.454501,22

In [372]:
s.write_file("EDHCSg.geojson", EDHCS_autotypes)

A file with the same name ("EDHCSg.geojson") already exists in this location.
Press Enter to overwrite it or choose different path and filename: 
Your <class 'geopandas.geodataframe.GeoDataFrame'> object has been succefully written as "https://sciencedata.dk/files/SDAM_root/SDAM_data/EDHCS/EDHCSg.geojson"


In [359]:
print(test_results_df.to_markdown(tablefmt="github"))

|    |   threshold (=/>) |   proportion |    N |   correct |
|----|-------------------|--------------|------|-----------|
|  0 |              0.3  |         1    | 4601 |  0.862421 |
|  1 |              0.35 |         0.94 | 4337 |  0.890247 |
|  2 |              0.4  |         0.93 | 4310 |  0.8942   |
|  3 |              0.45 |         0.91 | 4213 |  0.898647 |
|  4 |              0.5  |         0.9  | 4162 |  0.90197  |
|  5 |              0.55 |         0.89 | 4092 |  0.90567  |
|  6 |              0.6  |         0.86 | 3955 |  0.908976 |
|  7 |              0.65 |         0.84 | 3898 |  0.910723 |
|  8 |              0.7  |         0.71 | 3264 |  0.957414 |
|  9 |              0.75 |         0.67 | 3078 |  0.968811 |
| 10 |              0.8  |         0.58 | 2694 |  0.971047 |
| 11 |              0.85 |         0.51 | 2336 |  0.977312 |
| 12 |              0.9  |         0.42 | 1957 |  0.984159 |
| 13 |              0.95 |         0.34 | 1563 |  0.989763 |


In [361]:
print(model_results_df.to_markdown(tablefmt="github"))

|    |   threshold (=/>) |   proportion |     N |
|----|-------------------|--------------|-------|
|  0 |              0.3  |         0.99 | 82791 |
|  1 |              0.35 |         0.98 | 81618 |
|  2 |              0.4  |         0.97 | 81048 |
|  3 |              0.45 |         0.95 | 79585 |
|  4 |              0.5  |         0.95 | 78929 |
|  5 |              0.55 |         0.94 | 78289 |
|  6 |              0.6  |         0.93 | 77384 |
|  7 |              0.65 |         0.92 | 76863 |
|  8 |              0.7  |         0.73 | 61115 |
|  9 |              0.75 |         0.71 | 59070 |
| 10 |              0.8  |         0.61 | 50587 |
| 11 |              0.85 |         0.56 | 46450 |
| 12 |              0.9  |         0.49 | 40601 |
| 13 |              0.95 |         0.43 | 36118 |


# Automatic assigment of `type_of_inscription`

The EDHCS dataset consists of 137,305 inscriptions. These inscriptions might be classified as falling into certain inscription categories. However, the two source datasets (EDH and EDCS) employ a different approach to inscription classification. First of all, EDH uses English labels (e.g. "epitaph", "votive inscription" etc.), while EDCS uses Latin labels (e.g. "tituli_sepulcrales" or "tituli sacri"). In some cases, these types can be matched upon each other in a straightforward manner, in other cases this is not so simple. For instance, since the typological categories employed by EDCS are not mutually exclusive, some inscriptions are classified by means of five different labels. Furthermore, one category in the first source might correspond to several categories within the second source. For instance, the `owner/artist inscription` category in EDH corresponds either to `tituli possessionis` and `tituli fabricationis` in EDCS.

Since we attempt to make the dataset as a whole accessible to cross-category comparisons, we had to decide which one of the two classification systems to employ. We decided for the EDH categories, especially because they are mutually exclusive, what makes the comparison slighthly more straitforward. 

However, since the EDH labels are present only for inscriptions included in the EDH dataset, we had to come up with a solution how top assign them to inscriptions from the EDCS dataset as well. This is a typical **document classification task**, which might be solved by numourous supervised machine learning methods. Here we decided for [Support Vector Machine (SVM)](https://en.wikipedia.org/wiki/Support-vector_machine) model to be trained on a subset of inscriptions which are included both in EDCS and EDH (for the whole implementation, see `scripts/3_INSCR-TYPE-AUTO.ipynb`). 

From the total number of 137,305 inscriptions in the EDHCS dataset, 49,916 are shared by both the EDH and EDCS dataset. From this subset, 46,171 are classified by means of the `type_of_inscription_clean` attribute  in EDH (the remaining 3,745 inscriptions are classified as `NULL`).  To build the SVM model, this constrained subset was divided into *training* and *test* datasets (90% were used to train and 10% (4,617) to test the model).

The aim was to use some attributes from EDCS to classify the EDCS inscriptions according to the EDH inscription categories (e.g. as "epitaph", "votive inscription" etc.). After some experimenting, we realized that we can use information from at least three different columns:
* `status_list`, including information about the inscription category according to EDCS combined with some other metadata
* `Material`, containing information about the material from which the inscription is built
* `clean_text_interpretive_word`, a preprocessed text of the inscription

These three columns were used to construct 157 features to feed a TFIDF vectorizer. The features were formed by 100 most common bigrams in the texts of the inscriptions (e.g. "Dis_Manibus", "vixit_annos" or "votum_solvit"). Further, 37 were based on all unqique categories within the attribute `status_list`. Finally, 20 features were based on 20 most common categories from the `Material` attribute (e.g. "lapis", "opus_figlinae", "aes"). (Underscore was used to treat multi-word features as one word features).


### Model Testing

For testing of the model we used 4,617 labeled inscriptions, randomly selected from 46,171 inscriptions contained both in EDH and EDCS, which have a non-empty `type_of_inscription_clean` attribute and which have not been used to  train the model. 

Each prediction of the SVM model is accompanied by **probability** on scale {0-1}, expressing a level of certainty concerning the predicted classification category. Thus, for instance, the inscription `EDCS-44500113` is classified as `votive inscription` with 0.919633 probability. On closer inscpection, we see that EDCS classifies this inscription as `tituli sacri`, what is probably the most important feature driving the prediction in this case.

These probabilities might be used to formulate certain thresholds under which the classification will not be accepted. In Table 1, we see that 90 % of inscriptions in the test dataset has been classified with probability equal to- or higher than 0.5 and that this classification was correct in 90 % of cases. Further, approximately 71 % of inscriptions has been classified  with probability equal to- or higher than 0.7. From these more than 95 % was classified correctly. These results can inform us when we apply the model upon an unlabled dataset, where we can expect a comparable ratios between threshold values, proportions of covered inscriptions and the extent of correct classifications.


|    |   threshold (=/>) |   proportion |    N |   correct |
|----|-------------------|--------------|------|-----------|
|  0 |              0.3  |         1    | 4601 |  0.862421 |
|  1 |              0.35 |         0.94 | 4337 |  0.890247 |
|  2 |              0.4  |         0.93 | 4310 |  0.8942   |
|  3 |              0.45 |         0.91 | 4213 |  0.898647 |
|  4 |              0.5  |         0.9  | 4162 |  0.90197  |
|  5 |              0.55 |         0.89 | 4092 |  0.90567  |
|  6 |              0.6  |         0.86 | 3955 |  0.908976 |
|  7 |              0.65 |         0.84 | 3898 |  0.910723 |
|  8 |              0.7  |         0.71 | 3264 |  0.957414 |
|  9 |              0.75 |         0.67 | 3078 |  0.968811 |
| 10 |              0.8  |         0.58 | 2694 |  0.971047 |
| 11 |              0.85 |         0.51 | 2336 |  0.977312 |
| 12 |              0.9  |         0.42 | 1957 |  0.984159 |
| 13 |              0.95 |         0.34 | 1563 |  0.989763 |

After testing, both the model and the vectorizer used to preprocess the data were saved as pickles for future reuse as `data/svm_model_v1.0.sav` and `data/svm_vectorizer_v1.0.sav`. 

### Model application

After training and testing the model on labeled data, we proceeded to apply the model on 83,482 inscriptions which are exclusively in EDCS. In this case, we cannot directly measure the proportion of correctly classified inscriptions. But we can use the data from testing to make an estimate.

|    |   threshold (=/>) |   proportion |     N |
|----|-------------------|--------------|-------|
|  0 |              0.3  |         0.99 | 82791 |
|  1 |              0.35 |         0.98 | 81618 |
|  2 |              0.4  |         0.97 | 81048 |
|  3 |              0.45 |         0.95 | 79585 |
|  4 |              0.5  |         0.95 | 78929 |
|  5 |              0.55 |         0.94 | 78289 |
|  6 |              0.6  |         0.93 | 77384 |
|  7 |              0.65 |         0.92 | 76863 |
|  8 |              0.7  |         0.73 | 61115 |
|  9 |              0.75 |         0.71 | 59070 |
| 10 |              0.8  |         0.61 | 50587 |
| 11 |              0.85 |         0.56 | 46450 |
| 12 |              0.9  |         0.49 | 40601 |
| 13 |              0.95 |         0.43 | 36118 |


As a result of applying the model, we enriched the whole EDHCS dataset by 2 new attributes:
* `type_of_inscription_auto`, containing either the predicted label or - where available - the label from `type_of_inscription_clean` 
* `type_of_inscription_auto_prob`, expressing the probability on the scale from 0 to 1 (1 is used for datapoints where the `type_of_inscription_clean` was used)

When we look at the dataset as whole, we see that from the 137,305 inscriptions, 127,773 (93 %) are classified in `type_of_inscription_auto` with probability equal to- or higher than 0.55. Using the 0.55 threshold, this proportion of the dataset can be used for further comparisons.

In [374]:
# to train the model, we will work with a subset of the for which we have both the EDH and EDCS attributes
# we will call it EDHg
EDH_overlap_all = EDHCSg[(EDHCSg["EDH-ID"].notnull()) & (EDHCSg["EDCS-ID"].notnull())] 
EDH_overlap_all.groupby("type_of_inscription_clean").size().sort_values(ascending=False)

type_of_inscription_clean
epitaph                            21520
votive inscription                 11728
NULL                                3745
owner/artist inscription            3340
honorific inscription               3003
building/dedicatory inscription     2561
mile-/leaguestone                   1307
identification inscription           850
acclamation                          287
defixio                              269
list                                 246
military diploma                     209
label                                194
boundary inscription                 175
elogium                              132
letter                               119
public legal inscription             109
seat inscription                      42
private legal inscription             36
prayer                                18
assignation inscription               15
calendar                              10
adnuntiatio                            1
dtype: int64

In [376]:
EDHCS_autotypes.groupby("type_of_inscription_auto").size().sort_values(ascending=False)

type_of_inscription_auto
epitaph                            85259
votive inscription                 16338
owner/artist inscription           10677
honorific inscription               5344
building/dedicatory inscription     3634
mile-/leaguestone                   2798
identification inscription          1205
boundary inscription                 399
acclamation                          368
military diploma                     348
list                                 338
defixio                              298
label                                220
elogium                              141
public legal inscription             137
letter                               130
seat inscription                      46
private legal inscription             39
assignation inscription               21
prayer                                20
calendar                              11
adnuntiatio                            2
dtype: int64